Import pandas and read the dataset

In [1]:
import pandas as pd
d = pd.read_csv("./YouTubeSpamCollectionV1/Youtube01-Psy.csv")

class 1 = comment is spam; class 0 = comment is not spam

In [2]:
d.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [3]:
print("Spam comments: "+str(len(d.query("CLASS == 1"))))
print("Non-spam comments: "+str(len(d.query("CLASS == 0"))))
print("All comments: "+str(len(d)))

Spam comments: 175
Non-spam comments: 175
All comments: 350


import CountVectorizerfrom sklearn, so that we can count each occurrence of every word in the dataset.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

Fit the dataset into vectorizer and transform them into a bag of words.

In [5]:
dvec = vectorizer.fit_transform(d['CONTENT'])

Print and analyze a random comment

In [6]:
print(d["CONTENT"][284])
#analyze(d["CONTENT"][224])
#dvec.analyze(d["CONTENT"][284])

1 million dislikes!EPIC FAIL(ready for you fanboys)﻿


shufle the dataset and save it to dshuf

In [7]:
dshuf = d.sample(frac=1)

Separate the dataset into train and test datasets, and then into attributes and labels for each dataset.

"In the preceding code, d_train_att = vectorizer.fit_transform(d_train["CONTENT"]), is an
important step. At that stage, you have a training set that you want to perform a fit
transform on, which means it will learn the words and also produce the matrix. However,
for the testing set, we don't perform a fit transform again, since we don't want the model to
learn different words for the testing data." The book

In [8]:
d_train = dshuf[:300]
d_test = dshuf[300:]
d_train_att = vectorizer.fit_transform(d_train["CONTENT"]) # fit and transform on the training set
d_test_att = vectorizer.transform(d_test["CONTENT"]) # reuse it on the test set
d_train_label = d_train["CLASS"] 
d_test_label = d_test["CLASS"]  

Import RandomForestClassifier and create a new object with it

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)

Fit the training set to the clf (the RandomForestClassifier object)

In [10]:
clf.fit(d_train_att, d_train_label)

RandomForestClassifier(n_estimators=80)

Test the clf on the testing set

In [11]:
clf.score(d_test_att, d_test_label) #test

0.98

Create a confusion metrix

In [12]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)

array([[25,  0],
       [ 1, 24]])

Get the score of our model

In [13]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.94 (+/- 0.05)


Load all databses and combine them

In [14]:
d_all = pd.concat([
    pd.read_csv("./YouTubeSpamCollectionV1/Youtube01-Psy.csv"),
    pd.read_csv("./YouTubeSpamCollectionV1/Youtube02-KatyPerry.csv"),
    pd.read_csv("./YouTubeSpamCollectionV1/Youtube03-LMFAO.csv"),
    pd.read_csv("./YouTubeSpamCollectionV1/Youtube04-Eminem.csv"),
    pd.read_csv("./YouTubeSpamCollectionV1/Youtube05-Shakira.csv")
])

In [15]:
print("Spam comments: "+str(len(d_all.query("CLASS == 1"))))
print("Non-spam comments: "+str(len(d_all.query("CLASS == 0"))))
print("All comments: "+str(len(d_all)))

Spam comments: 1005
Non-spam comments: 951
All comments: 1956


Setting up databeses

In [16]:
d_all_shuf = d_all.sample(frac=1)
d_all_content = d_all_shuf['CONTENT']
d_all_label = d_all_shuf['CLASS']

Setting up a sklearn Pipeline

In [17]:
from sklearn.pipeline import Pipeline, make_pipeline
pipeline = Pipeline([
    ("bag-of-words", CountVectorizer()),
    ("random forest", RandomForestClassifier()),
])
print(pipeline)

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])


"We can let the pipeline name of each step by itself by adding CountVectorizer in
our RandomForestClassifier and it will name
them CountVectorizer and RandomForestClassifier" The book

In [18]:
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

Fit the pipeline

In [19]:
pipeline.fit(d_all_content[:1500], d_all_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

Calculate the score of the pipeline using the rest of the data

In [20]:
pipeline.score(d_all_content[1500:], d_all_label[1500:])

0.9671052631578947

Manually test the pipeline on some examples. (still 1 = spam, 0 = no spam)

In [21]:
print(pipeline.predict(['plz subscribe to my channel'])) # should say 1
print(pipeline.predict(['damn, I loved when you drank the water'])) # should say 0

[1]
[0]


Get the score of our pipeline

In [22]:
scores = cross_val_score(pipeline, d_all_content, d_all_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.01)


Add TF-IDF to a new pipeline

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(), TfidfTransformer(norm=None), RandomForestClassifier())

Get the score of our new pipeline

In [24]:
scores = cross_val_score(pipeline2, d_all_content, d_all_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.01)


Show the steps we took to create the second pipeline

In [25]:
pipeline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]

Adding all possible parameters to a variable, then using GridSearchCV to find the best parameters combination

In [26]:
parameters = {
    "countvectorizer__max_features":(None, 1000, 2000),
    "countvectorizer__ngram_range":((1, 1), (1, 2)), # unigrams or bigrams
    "countvectorizer__stop_words":('english', None),
    "tfidftransformer__use_idf":(True, False), # turn on/off tfidf
    "randomforestclassifier__n_estimators":(20, 50, 100)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=1, verbose=1)

"Using ngrams, we can mention just single words or pairs of words that are stop words, use
the English dictionary of stop words, or don't use stop words, which means in the first case
we need to get rid of common words, and in the second case we do not get rid of common
words. Using TF-IDF, we use JEG to state whether it's yes or no. The random forest we
created uses 20, 50, or 100 trees. Using this, we can perform a grid search, which runs
through all of the combinations of parameters and finds out what the best combination is.
So, let's give our pipeline number 2, which has the TF-IDF along with it. We will use fit to
perform the search" The book

In [27]:
grid_search.fit(d_all_content, d_all_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

Get the score of our fit model (haha, see what I did there?)

In [28]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set\combination:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.963
Best parameters set\combination:
	countvectorizer__max_features: 2000
	countvectorizer__ngram_range: (1, 2)
	countvectorizer__stop_words: 'english'
	randomforestclassifier__n_estimators: 50
	tfidftransformer__use_idf: True
